<a href="https://colab.research.google.com/github/badzlan/techfest-elims/blob/main/EDA_tipiel_IPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Efektivitas Penyaluran Dana Transfer ke Daerah (TKDD): Studi Perbandingan Pagu dan Realisasi TKDD di Provinsi-Provinsi Indonesia Tahun 2023

## 1. Data Preparation

### 1.1 Import data

In [3]:
import pandas as pd

# Baca data dari GitHub
url = "https://raw.githubusercontent.com/badzlan/techfest-elims/refs/heads/main/dataset.csv"
df = pd.read_csv(url)

# Tampilkan 5 data pertama
df.head()

,Provinsi,IPM,Pagu TKDD,Realisasi TKDD,Jumlah Penduduk,Anggaran APBN per kapita,Persentase Penduduk Miskin,Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,Produk Domestik Regional Bruto per Kapita HB,Laju Pertumbuhan PDRB atas dasar konstan 2010
0,ACEH,"74,7","Rp 30.821.480.000.000,00","Rp 31.074.560.000.000,00",54.092.000,"574.476,08","14,45","227018,2",41.408.000,"4,23"
1,BALI,"78,01","Rp 11.152.110.000.000,00","Rp 11.064.980.000.000,00",43.743.000,"252.954,30","4,25","274358,18",62.294.000,"5,71"
2,BANTEN,"75,77","Rp 17.453.380.000.000,00","Rp 17.467.380.000.000,00",12.167.000,"1.435.635,74","6,17","814122,03",66.147.000,"4,81"
3,BENGKULU,"74,3","Rp 10.570.080.000.000,00","Rp 10.559.790.000.000,00",20.594.000,"512.760,51","14,04","96583,09",46.300.000,"4,28"
4,DI YOGYAKARTA,"81,09","Rp 10.355.870.000.000,00","Rp 10.241.290.000.000,00",37.126.000,"275.852,23","11,04","180696,04",48.360.000,"5,07"


### 1.2 Cek Missing Values & Duplikasi

In [4]:
# Cek missing values
print("Jumlah missing value per kolom:")
print(df.isnull().sum())

# Cek duplikasi
print("\nJumlah baris duplikat:", df.duplicated().sum())

Jumlah missing value per kolom:
Provinsi                                                          0
IPM                                                               4
Pagu TKDD                                                         0
Realisasi TKDD                                                    0
Jumlah Penduduk                                                   2
Anggaran APBN per kapita                                          0
Persentase Penduduk Miskin                                        4
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku    0
Produk Domestik Regional Bruto per Kapita HB                      0
Laju Pertumbuhan PDRB atas dasar konstan 2010                     0
dtype: int64

Jumlah baris duplikat: 0


### 1.3 Cek & Pastikan Tipe Data Numerik

In [5]:
# Cek tipe data
print("\nTipe data setiap kolom:")
print(df.dtypes)

# Jika ada kolom numerik dalam bentuk string (object), ubah ke float
kolom_numerik = [
    'IPM',
    'Pagu TKDD',
    'Realisasi TKDD',
    'Jumlah Penduduk',
    'Anggaran APBN per kapita',
    'Persentase Penduduk Miskin',
    'Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku',
    'Produk Domestik Regional Bruto per Kapita HB',
    'Laju Pertumbuhan PDRB atas dasar konstan 2010'
]

# Bersihkan karakter non-numerik dan ubah menjadi float
for kolom in kolom_numerik:
    df[kolom] = df[kolom].replace({',': '', 'Rp': '', '%': '', ' ': ''}, regex=True)
    df[kolom] = pd.to_numeric(df[kolom], errors='coerce')

# Cek hasil konversi
df[kolom_numerik].dtypes


Tipe data setiap kolom:
Provinsi                                                          object
IPM                                                               object
Pagu TKDD                                                         object
Realisasi TKDD                                                    object
Jumlah Penduduk                                                   object
Anggaran APBN per kapita                                          object
Persentase Penduduk Miskin                                        object
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku    object
Produk Domestik Regional Bruto per Kapita HB                      object
Laju Pertumbuhan PDRB atas dasar konstan 2010                     object
dtype: object


,0
IPM,float64
Pagu TKDD,float64
Realisasi TKDD,float64
Jumlah Penduduk,float64
Anggaran APBN per kapita,float64
Persentase Penduduk Miskin,float64
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,int64
Produk Domestik Regional Bruto per Kapita HB,float64
Laju Pertumbuhan PDRB atas dasar konstan 2010,int64


### 1.4 Normalisasi Kolom

In [6]:
# Contoh normalisasi: TKDD per penduduk
df['Realisasi TKDD per Kapita'] = df['Realisasi TKDD'] / df['Jumlah Penduduk']
df['Pagu TKDD per Kapita'] = df['Pagu TKDD'] / df['Jumlah Penduduk']


### 1.5 Tambahkan Kolom “Rasio Realisasi TKDD”

In [7]:
df['Rasio Realisasi (%)'] = (df['Realisasi TKDD'] / df['Pagu TKDD']) * 100


### 1.6 Klasifikasikan Rasio ke dalam 3 Kategori

In [8]:
def klasifikasi_realisasi(rasio):
    if rasio < 90:
        return 'Di bawah 90%'
    elif 90 <= rasio <= 100:
        return '90 - 100%'
    else:
        return 'Di atas 100%'

df['Kategori Realisasi'] = df['Rasio Realisasi (%)'].apply(klasifikasi_realisasi)


In [ ]:
df = data.set_index('Provinsi')
df.sample(3)

,IPM,Pagu TKDD,Realisasi TKDD,Jumlah Penduduk,Anggaran APBN per kapita,Persentase Penduduk Miskin,Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,Produk Domestik Regional Bruto per Kapita HB,Laju Pertumbuhan PDRB atas dasar konstan 2010
Provinsi,,,,,,,,,
KALIMANTAN TENGAH,"73,73","Rp 27.690.230.000.000,00","Rp 28.862.210.000.000,00",27.372.000,"1.054.442,86","5,11","208783,35",75.271.000,"4,14"
BANTEN,"75,77","Rp 17.453.380.000.000,00","Rp 17.467.380.000.000,00",12.167.000,"1.435.635,74","6,17","814122,03",66.147.000,"4,81"
PAPUA PEGUNUNGAN,NaN,"Rp 13.543.140.000.000,00","Rp 12.821.760.000.000,00",NaN,null,NaN,"24433,99",16.870.000,"4,78"


In [ ]:
# Cek Missing Values
df.isnull().sum()

,0
IPM,4
Pagu TKDD,0
Realisasi TKDD,0
Jumlah Penduduk,2
Anggaran APBN per kapita,0
Persentase Penduduk Miskin,4
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,0
Produk Domestik Regional Bruto per Kapita HB,0
Laju Pertumbuhan PDRB atas dasar konstan 2010,0


In [ ]:
# Showing Data Type from Columns
df.dtypes

,0
IPM,object
Pagu TKDD,object
Realisasi TKDD,object
Jumlah Penduduk,object
Anggaran APBN per kapita,object
Persentase Penduduk Miskin,object
Produk Domestik Regional Bruto (PDRB) Atas Dasar Harga Berlaku,object
Produk Domestik Regional Bruto per Kapita HB,object
Laju Pertumbuhan PDRB atas dasar konstan 2010,object
